# Эксперименты с форматом brat

In [124]:
from typing import List, Tuple, Iterable
from pathlib import Path
import nltk
from collections import namedtuple
from nltk.tokenize import WordPunctTokenizer, RegexpTokenizer, PunktSentenceTokenizer

In [ ]:
brat_dir = Path('/home/max/datasets/RuREBus_data/brat')
brat_doc_name = '20103011022200910379001_0_part_0'
brat_text_path = (brat_dir / brat_doc_name).with_suffix('.txt')
brat_ann_path = brat_text_path.with_suffix('.ann')

In [ ]:
text = brat_text_path.open().read()

In [ ]:
print(text[:200])

In [205]:
Entity = namedtuple('Entity', 'id label position text')
Position = namedtuple('Position', 'start end')

def read_annotation(path: Path) -> List[Entity]:
    entities = [Entity(id_, label, Position(int(start), int(end)), phrase) for id_, label, start, end, phrase in
                ((id_, *part.split(), phrase.rstrip()) for id_, part, phrase in
                (s.split('\t') for s in path.open() if s.startswith('T')))]
    return sorted(entities, key=lambda e: e.position.start)

In [206]:
annotation = read_annotation(brat_ann_path)

In [207]:
annotation[:10]

[Entity(id='T2', label='BIN', position=Position(start=26, end=37), text='УТВЕРЖДЕНИИ'),
 Entity(id='T3', label='BIN', position=Position(start=73, end=81), text='РАЗВИТИЯ'),
 Entity(id='T6', label='BIN', position=Position(start=112, end=118), text='Принят'),
 Entity(id='T7', label='BIN', position=Position(start=211, end=220), text='Утвердить'),
 Entity(id='T8', label='BIN', position=Position(start=268, end=276), text='развития'),
 Entity(id='T10', label='BIN', position=Position(start=334, end=349), text='вступает в силу'),
 Entity(id='T14', label='BIN', position=Position(start=509, end=520), text='утверждении'),
 Entity(id='T16', label='BIN', position=Position(start=631, end=639), text='РАЗВИТИЯ'),
 Entity(id='T20', label='BIN', position=Position(start=756, end=771), text='входит в состав'),
 Entity(id='T22', label='MET', position=Position(start=825, end=848), text='Площадь территории края')]

In [ ]:
_, _, (start, end), phrase = annotation[0]

In [ ]:
assert text[start: end] == phrase

In [ ]:
text[start: end]

# Разбиваем текст на предложения и токены.

In [ ]:
sent_detector = nltk.data.load('tokenizers/punkt/russian.pickle')

In [233]:
test_text = 'У попа была "собака",\nон ее любил. Она съела. Он ее убил. Kek!'

In [97]:
list(sent_detector.span_tokenize_sents(test_text))

[[(0, 32), (33, 43), (44, 55)], [], [(0, 4)]]

In [214]:
test_text[44:55]

[]

In [237]:
import re, string

In [117]:
def tokenize_text(sent_tokenizer: PunktSentenceTokenizer, 
                  word_tokenizer: RegexpTokenizer,
                  text: str) -> Iterable[List[Tuple[int, int]]]:
    paragraphs = text.split('\n')
    para_sents = list(sent_tokenizer.span_tokenize_sents(paragraphs))
    parargraph_start = 0
    for para_sents, para_text in zip(para_sents, paragraphs):
        for sent_start, sent_end in para_sents:
            sentence = []
            sent_text = text[parargraph_start + sent_start: parargraph_start + sent_end]
            for token_start, token_end in word_tokenizer.span_tokenize(sent_text):
                offset = parargraph_start + sent_start
                sentence.append((offset + token_start, offset + token_end))
            yield sentence
        parargraph_start += len(para_text) + 1

In [239]:
sentences = list(tokenize_text(sent_detector, RegexpTokenizer(f'\w+|[{re.escape(string.punctuation)}]|\S+'), text))

# Строим IOB

In [240]:
def build_iob(
    text: str,
    sentences: Iterable[List[Tuple[int, int]]],
    annotation: List[Entity]) -> Iterable[str]:
    entity_index = 0
    entity = annotation[entity_index] if annotation else None
    label = None
    entity_started = False
    for sent in sentences:
        for start, end in sent:
            if entity and start >= entity.position.end:
                entity_index += 1
                entity = annotation[entity_index] if len(annotation) > entity_index else None
                entity_started = False
            if entity and start >= entity.position.start and end <= entity.position.end:
                if not entity_started and label == entity.label:
                    prefix = 'B'
                else:
                    prefix = 'I'
                label = entity.label
                tag = prefix + '-' + label
                entity_started = True
            else:
                label = 'O'
                tag = label
            yield '\t'.join(map(str, (text[start: end], tag, start, end)))
        yield ''

In [241]:
for s in build_iob(text, sentences, annotation):
    print(s)

АЛТАЙСКИЙ	O	0	9
КРАЙ	O	10	14

ЗАКОН	O	16	21

ОБ	O	23	25
УТВЕРЖДЕНИИ	I-BIN	26	37
СТРАТЕГИИ	O	38	47
СОЦИАЛЬНО	O	48	57
-	O	57	58
ЭКОНОМИЧЕСКОГО	O	58	72
РАЗВИТИЯ	I-BIN	73	81

АЛТАЙСКОГО	O	82	92
КРАЯ	O	93	97
ДО	O	98	100
2025	O	101	105
ГОДА	O	106	110

Принят	I-BIN	112	118

Постановлением	O	119	133
Алтайского	O	134	144
краевого	O	145	153

Законодательного	O	154	170
Собрания	O	171	179

от	O	180	182
19	O	183	185
.	O	185	186
11	O	186	188
.	O	188	189
2012	O	189	193
N	O	194	195
569	O	196	199

Статья	O	201	207
1	O	208	209

Утвердить	I-BIN	211	220
прилагаемую	O	221	232
стратегию	O	233	242
социально	O	243	252
-	O	252	253
экономического	O	253	267
развития	I-BIN	268	276
Алтайского	O	277	287
края	O	288	292
до	O	293	295
2025	O	296	300
года	O	301	305
.	O	305	306

Статья	O	308	314
2	O	315	316

Настоящий	O	318	327
Закон	O	328	333
вступает	I-BIN	334	342
в	I-BIN	343	344
силу	I-BIN	345	349
со	O	350	352
дня	O	353	356
его	O	357	360
официального	O	361	373
опубликования	O	374	387
.	O	387	388

Губернатор	O	390	400


средней	I-MET	14357	14364
продолжительности	I-MET	14365	14382
жизни	I-MET	14383	14388
.	O	14388	14389

В	O	14390	14391
рейтинге	O	14392	14400
регионов	I-INST	14401	14409
Сибирского	I-INST	14410	14420
федерального	I-INST	14421	14433
округа	I-INST	14434	14440
по	O	14441	14443
продолжительности	I-MET	14444	14461
жизни	I-MET	14462	14467
Алтайский	I-INST	14468	14477
край	I-INST	14478	14482
уступает	O	14483	14491
только	O	14492	14498
Новосибирской	I-INST	14499	14512
и	I-INST	14513	14514
Омской	I-INST	14515	14521
областям	I-INST	14522	14530
.	O	14530	14531

За	O	14532	14534
последние	O	14535	14544
годы	O	14545	14549
в	O	14550	14551
Алтайском	I-INST	14552	14561
крае	I-INST	14562	14566
активное	I-QUA	14567	14575
развитие	I-CMP	14576	14584
получила	I-BIN	14585	14593
социальная	I-SOC	14594	14604
сфера	I-SOC	14605	14610
,	O	14610	14611
значительно	I-QUA	14612	14623
обновилась	I-BIN	14624	14634
ее	O	14635	14637
материальная	O	14638	14650
база	O	14651	14655
.	O	14655	14656

За	O	14657	14659
период	O

-	O	25502	25503
-	O	25503	25504
-	O	25504	25505
-	O	25505	25506
-	O	25506	25507
-	O	25507	25508
-	O	25508	25509
-	O	25509	25510
-	O	25510	25511
-	O	25511	25512
-	O	25512	25513
-	O	25513	25514
-	O	25514	25515
-	O	25515	25516
-	O	25516	25517
-	O	25517	25518

<	O	25519	25520
1	O	25520	25521
>	O	25521	25522
По	O	25523	25525
данным	O	25526	25532
обследования	I-ACT	25533	25545
населения	I-ACT	25546	25555
по	I-ACT	25556	25558
проблемам	I-ACT	25559	25568
занятости	I-ACT	25569	25578
.	O	25578	25579

В	O	25581	25582
Алтайском	I-INST	25583	25592
крае	I-INST	25593	25597
имеется	I-BIN	25598	25605
достаточно	O	25606	25616
мощный	I-QUA	25617	25623
потенциал	O	25624	25633
промышленного	I-ECO	25634	25647
производства	I-ECO	25648	25660
,	O	25660	25661
который	O	25662	25669
отличается	O	25670	25680
диверсифицированной	O	25681	25700
структурой	O	25701	25711
и	O	25712	25713
низкой	I-QUA	25714	25720
долей	I-MET	25721	25726
добывающей	I-MET	25727	25737
промышленности	I-MET	25738	25752
(	O	25753	25754
2	O	257

руд	I-ACT	36585	36588
,	O	36588	36589
введены	O	36590	36597
в	O	36598	36599
эксплуатацию	O	36600	36612
Рубцовский	I-INST	36613	36623
и	I-INST	36624	36625
Зареченский	I-INST	36626	36637
горнообогатительные	I-INST	36638	36657
комбинаты	I-INST	36658	36667
.	O	36667	36668

В	O	36669	36670
ближайшей	O	36671	36680
перспективе	O	36681	36692
будет	O	36693	36698
завершено	O	36699	36708
строительство	I-BIN	36709	36722
рудника	I-ECO	36723	36730
на	O	36731	36733
крупнейшем	I-QUA	36734	36744
Корбалихинском	I-ECO	36745	36759
месторождении	I-ECO	36760	36773
.	O	36773	36774

С	O	36775	36776
2010	O	36777	36781
года	O	36782	36786
в	O	36787	36788
крае	O	36789	36793
начата	O	36794	36800
промышленная	I-ACT	36801	36813
добыча	I-ACT	36814	36820
золота	I-ACT	36821	36827
.	O	36827	36828

ООО	I-INST	36829	36832
"	I-INST	36833	36834
Золото	I-INST	36834	36840
Курьи	I-INST	36841	36846
"	I-INST	36846	36847
ведет	O	36848	36853
разработку	I-BIN	36854	36864
Новофирсовского	I-ECO	36865	36880
золото	I-ECO	36881	36887
ру

-	I-ECO	47283	47284
Змеиногорский	I-ECO	47284	47297
горнорудный	I-ECO	47298	47309
комплекс	I-ECO	47310	47318
,	I-ECO	47318	47319
многочисленные	I-ECO	47320	47334
курганы	I-ECO	47335	47342
и	I-ECO	47343	47344
др	I-ECO	47345	47347
.	I-ECO	47347	47348
)	I-ECO	47348	47349
;	O	47349	47350
базовая	O	47351	47358
инфраструктура	I-ECO	47359	47373
для	I-ECO	47374	47377
культурного	I-ECO	47378	47389
туризма	I-ECO	47390	47397
,	O	47397	47398
связанная	O	47399	47408
с	O	47409	47410
проведением	I-ACT	47411	47422
событийных	I-ACT	47423	47433
мероприятий	I-ACT	47434	47445
межрегионального	I-ACT	47446	47462
значения	I-ACT	47463	47471
(	I-ACT	47472	47473
Шукшинские	I-ACT	47473	47483
дни	I-ACT	47484	47487
,	I-ACT	47487	47488
фестивали	I-ACT	47489	47498
"	I-ACT	47499	47500
Песни	I-ACT	47500	47505
иткульского	I-ACT	47506	47517
лета	I-ACT	47518	47522
"	I-ACT	47522	47523
и	I-ACT	47524	47525
т	I-ACT	47526	47527
.	I-ACT	47527	47528
д	I-ACT	47528	47529
.	I-ACT	47529	47530
)	I-ACT	47530	47531
,	O	47531	47532
а	O

In [191]:
def iob_to_brat(conll: Iterable[str], text: str) -> Iterable[Entity]:
    entity_id = 1
    entity = None
    for s in conll:
        if not s:
            continue
        token, label, start_str, end_str = s.split('\t')
        start, end = int(start_str), int(end_str)
        if entity:
            if label == 'O':
                yield entity
                entity = None
            elif label.startswith('B-') or label[2:] != entity.label:
                yield entity
                entity = Entity(f'T{entity_id}', label[2:], Position(start, end), text[start: end])
                entity_id += 1
            else:
                assert label == 'I-' + entity.label
                entity = Entity(entity.id, 
                                entity.label, 
                                Position(entity.position.start, end), 
                                text[entity.position.start: end])
            continue
        if label == 'O':
            continue
        entity = Entity(f'T{entity_id}', label[2:], Position(start, end), text[start: end])
        entity_id += 1
    if entity:
        yield entity

In [189]:
'T25	MET 907 948	Средне годовая численность населения края'

'T25\tMET 907 948\tСредне годовая численность населения края'

In [242]:
annotation_new = list(iob_to_brat(build_iob(text, sentences, annotation), text))

In [246]:
for i, (e1, e2) in enumerate(zip(annotation, annotation_new)):
    if e1.label != e2.label:
        print(e1, e2)
    if e1.text != e2.text:
        print(e1, e2)

Entity(id='T917', label='QUA', position=Position(start=32454, end=32471), text='один из крупнейши') Entity(id='T887', label='QUA', position=Position(start=32454, end=32461), text='один из')
Entity(id='T1203', label='ECO', position=Position(start=44532, end=44612), text='общий сайт в сети Интернет "Алтай транс граничный" (http://www.altaiinter.ECOo/)') Entity(id='T1173', label='ECO', position=Position(start=44532, end=44612), text='общий сайт в сети Интернет "Алтай транс граничный" (http://www.altaiinter.info/)')
Entity(id='T1456', label='ECO', position=Position(start=56552, end=56577), text='пассажирскими авиа рейсам') Entity(id='T1426', label='ECO', position=Position(start=56552, end=56570), text='пассажирскими авиа')


In [244]:
annotation_new[886]

Entity(id='T887', label='QUA', position=Position(start=32454, end=32461), text='один из')

In [245]:
annotation[886]

Entity(id='T917', label='QUA', position=Position(start=32454, end=32471), text='один из крупнейши')

In [247]:
text[56552: 56577]

'пассажирскими авиа рейсам'